In [2]:
#!/usr/bin/env python
# coding:utf-8

import os
import json
from py2neo import Graph, Node


class MedicalGraph:
    # 图数据库初始化设置
    def __init__(self):
        cur_dir = '/'.join(os.path.abspath('__file__').split('//')[:-1])
        self.data_path = os.path.join(cur_dir, '/Users/xieshenqiu/Desktop/medical.json')
        # 配置Neo4j
        self.g = Graph(
            host='127.0.0.1',
            http_port=7474,
            user='neo4j',
            password='123456')

    # 读取节点
    def read_nodes(self):
        # 共8类节点
        drugs = []  # 药品
        foods = []  # 食物
        checks = []  # 检查
        departments = []  # 科室
        producers = []  # 药品大类
        diseases = []  # 疾病
        symptoms = []  # 症状
        disease_infos = []  # 疾病信息

        # 构建节点实体关系
        rels_department = []  # 科室-科室关系
        rels_noteat = []  # 疾病-忌吃食物关系
        rels_doeat = []  # 疾病-宜吃食物关系
        rels_recommandeat = []  # 疾病－推荐吃食物关系
        rels_commonddrug = []  # 疾病－通用药品关系
        rels_recommanddrug = []  # 疾病－热门药品关系
        rels_check = []  # 疾病-检查关系
        rels_drug_producer = []  # 厂商-药物关系

        rels_symptom = []  # 疾病症状关系
        rels_acompany = []  # 疾病并发关系
        rels_category = []  # 疾病与科室之间的关系

        count = 0
        file = open(self.data_path, encoding='utf-8')
        data = file.readline()
        while data:
            disease_dict = {}
            count += 1
            print(count)
            # 根据json文件加载进来的数据
            data_json = json.loads(data)
            disease = data_json['name']
            disease_dict['name'] = disease
            disease_dict['desc'] = ''
            disease_dict['prevent'] = ''
            disease_dict['cause'] = ''
            disease_dict['easy_get'] = ''
            disease_dict['cure_department'] = ''
            disease_dict['cure_way'] = ''
            disease_dict['cure_lasttime'] = ''
            disease_dict['symptom'] = ''
            disease_dict['cured_prob'] = ''

            # 构建疾病和症状的关系
            if 'symptom' in data_json:
                symptoms += data_json['symptom']
                disease_dict['symptom'] = data_json['symptom']
                for symptom in data_json['symptom']:
                    rels_symptom.append([disease, symptoms])

            # 构建疾病和并发症关系
            if 'acompany' in data_json:
                for acompany in data_json['acompany']:
                    rels_acompany.append([disease, acompany])

            # 存储疾病描述
            if 'desc' in data_json:
                disease_dict['desc'] = data_json['desc']

            # 存储疾病预防
            if 'prevent' in data_json:
                disease_dict['prevent'] = data_json['prevent']

            # 存储疾病成因
            if 'cause' in data_json:
                disease_dict['cause'] = data_json['cause']

            # 存储患病比例
            if 'get_prob' in data_json:
                disease_dict['get_prob'] = data_json['get_prob']

            # 存储易感人群
            if 'easy_get' in data_json:
                disease_dict['easy_get'] = data_json['easy_get']

            # 存储就诊科室
            if 'cure_department' in data_json:
                cure_department = data_json['cure_department']
                # 只有一个就诊科室
                if len(cure_department) == 1:
                    rels_category.append([disease, cure_department[0]])
                # 有两个就诊科室
                if len(cure_department) == 2:
                    big = cure_department[0]
                    small = cure_department[1]
                    # 构建科室间的关系
                    rels_department.append([small, big])
                    # 构建疾病与科室之间的关系
                    rels_category.append([disease, small])
                disease_dict['cure_department'] = cure_department
                departments += cure_department

            # 存储治疗方式
            if 'cure_way' in data_json:
                disease_dict['cure_way'] = data_json['cure_way']

            # 存储治疗周期
            if 'cure_lasttime' in data_json:
                disease_dict['cure_lasttime'] = data_json['cure_lasttime']

            # 存储治愈率
            if 'cured_prob' in data_json:
                disease_dict['cured_prob'] = data_json['cured_prob']

            # 存储常用药物
            if 'common_drug' in data_json:
                common_drug = data_json['common_drug']
                # 构建疾病和常用药物之间的关系
                for drug in common_drug:
                    rels_commonddrug.append([disease, drug])
                # 加入药品类
                drugs += common_drug

            # 存储药品推荐
            if 'recommand_drug' in data_json:
                recommand_drug = data_json['recommand_drug']
                # 加入药品类
                drugs += recommand_drug
                for drug in recommand_drug:
                    # 构建疾病和推荐药品之间的关系
                    rels_recommanddrug.append([disease, drug])

            # 存储忌食食物
            if 'not_eat' in data_json:
                not_eat = data_json['not_eat']
                # 构建忌食食物和疾病间的关系
                for _not in not_eat:
                    rels_noteat.append([disease, _not])
                # 食物中加入忌食食物
                foods += not_eat

            # 存储宜食食物
            if 'do_eat' in data_json:
                do_eat = data_json['do_eat']
                for _do in do_eat:
                    # 构建宜食食物和疾病的关系
                    rels_doeat.append([disease, _do])
                # 食物中加入宜食食物
                foods += do_eat

            # 存储推荐食物
            try:
                if 'recommand_eat' in data_json['recommand_eat']:
                    recommand_eat = data_json['recommand_eat']
                    for _recommand in recommand_eat:
                        # 构建推荐食物和疾病间的关系
                        rels_recommandeat.append([disease, _recommand])
                    # 食物中加入推荐食物
                    foods += recommand_eat
            except Exception as e:
                print(e)

            # 存储检查结果
            if 'check' in data_json:
                check = data_json['check']
                for _check in check:
                    # 构建疾病和检查的关系
                    rels_check.append([disease, _check])
                checks += check

            # 存储药品明细
            if 'drug_detail' in data_json:
                drug_detail = data_json['drug_detail']
                producer = [i.split('(')[0] for i in drug_detail]
                # 构建厂商-药物之间的关系
                rels_drug_producer += [[i.split('(')[0], i.split('(')[-1].replace(')', '')] for i in drug_detail]
                # 加入厂商名称
                producers += producer
            # 存入疾病信息
            disease_infos.append(disease_dict)
            data = file.readline()
        return set(drugs), set(foods), set(checks), set(departments), set(producers), set(symptoms), set(
            diseases), disease_infos, \
               rels_check, rels_recommandeat, rels_noteat, rels_doeat, rels_department, rels_commonddrug, rels_drug_producer, rels_recommanddrug, \
               rels_symptom, rels_acompany, rels_category

    # 建立节点
    def create_node(self, label, nodes):
        count = 0
        for node_name in nodes:
            node = Node(label, name=node_name)
            self.g.create(node)
            count += 1
            print(count, len(nodes))
        return

    # 创建知识图谱中心疾病的节点
    def create_diseases_nodes(self, disease_infos):
        count = 0
        for disease_dict in disease_infos:
            node = Node("Disease", name=disease_dict['name'], desc=disease_dict['desc'],
                        prevent=disease_dict['prevent'], cause=disease_dict['cause'],
                        easy_get=disease_dict['easy_get'], cure_lasttime=disease_dict['cure_lasttime'],
                        cure_department=disease_dict['cure_department']
                        , cure_way=disease_dict['cure_way'], cured_prob=disease_dict['cured_prob'])
            self.g.create(node)
            count += 1
            print(count)

    # 创建知识图谱实体节点类型schema
    def create_graphnodes(self):
        # 获取数据
        Drugs, Foods, Checks, Departments, Producers, Symptoms, Diseases, disease_infos, \
        rels_check, rels_recommandeat, rels_noteat, rels_doeat, rels_department, \
        rels_commonddrug, rels_drug_producer, rels_recommanddrug, rels_symptom, \
        rels_acompany, rels_category = self.read_nodes()
        # 创建疾病节点
        self.create_diseases_nodes(disease_infos)
        # 创建药品节点
        self.create_node('Drug', Drugs)
        # 输出药品的数量
        print(len(Drugs))
        # 创建食物节点
        self.create_node('Food', Foods)
        # 输出食物大小
        print(len(Foods))
        # 创建检查节点
        self.create_node('Check', Checks)
        # 输出检查节点数量
        print(len(Checks))
        # 创建科室节点
        self.create_node('Department', Departments)
        # 输出科室节点数量
        print(len(Departments))
        # 输出厂商节点
        self.create_node('Producer', Producers)
        # 输出厂商节点数量
        print(len(Producers))
        # 创建症状节点
        self.create_node('Symptom', Symptoms)
        return

    # 创建实体关联边
    def create_relationship(self, start_node, end_node, edges, rel_type, rel_name):
        '''
        :param start_node: 开始节点
        :param end_node: 结束节点
        :param edges: 关系边
        :param rel_type: 关系类型
        :param rel_name: 关系名称
        :return:
        '''
        count = 0
        # 去重处理
        set_edges = []
        for edge in edges:
            set_edges.append('###'.join(edge))
        all = len(set(set_edges))
        for edge in set(set_edges):
            edge = edge.split('###')
            p = edge[0]
            q = edge[1]
            # neo4j语法
            query = "match(p:%s),(q:%s) where p.name='%s'and q.name='%s' create (p)-[rel:%s{name:'%s'}]->(q)" % (
                start_node, end_node, p, q, rel_type, rel_name)
            try:
                self.g.run(query)
                count += 1
                print(rel_type, count, all)
            except Exception as e:
                print(e)
        return

    # 创建实体关系边
    def create_graphrels(self):
        Drugs, Foods, Checks, Departments, Producers, Symptoms, Diseases, disease_infos, rels_check, \
        rels_recommandeat, rels_noteat, rels_doeat, rels_department, rels_commonddrug, \
        rels_drug_producer, rels_recommanddrug, rels_symptom, rels_acompany, \
        rels_category = self.read_nodes()
        self.create_relationship('Disease', 'Food', rels_recommandeat, 'recommand_eat', '推荐食谱')
        self.create_relationship('Disease', 'Food', rels_noteat, 'no_eat', '忌吃')
        self.create_relationship('Disease', 'Food', rels_doeat, 'do_eat', '宜吃')
        self.create_relationship('Department', 'Department', rels_department, 'belongs_to', '属于')
        self.create_relationship('Disease', 'Drug', rels_commonddrug, 'common_drug', '常用药品')
        self.create_relationship('Producer', 'Drug', rels_drug_producer, 'drugs_of', '生产药品')
        self.create_relationship('Disease', 'Drug', rels_recommanddrug, 'recommand_drug', '好评药品')
        self.create_relationship('Disease', 'Check', rels_check, 'need_check', '诊断检查')
        self.create_relationship('Disease', 'Symptom', rels_symptom, 'has_symptom', '症状')
        self.create_relationship('Disease', 'Disease', rels_acompany, 'acompany_with', '并发症')
        self.create_relationship('Disease', 'Department', rels_category, 'belongs_to', '所属科室')

    # 导出数据
    def export_data(self):
        Drugs, Foods, Checks, Departments, Producers, Symptoms, Diseases, disease_infos, \
        rels_check, rels_recommandeat, rels_noteat, rels_doeat, rels_department, rels_commonddrug, \
        rels_drug_producer, rels_recommanddrug, rels_symptom, rels_acompany, \
        rels_category = self.read_nodes()
        f_drug = open('/Users/xieshenqiu/Downloads/知识图谱论文集/中医\ 问答系统/QASystemOnMedicalKG-master/dict/drug.txt', 'w+')
        f_food = open('/Users/xieshenqiu/Downloads/知识图谱论文集/中医\ 问答系统/QASystemOnMedicalKG-master/dict/food.txt', 'w+')
        f_check = open('/Users/xieshenqiu/Downloads/知识图谱论文集/中医\ 问答系统/QASystemOnMedicalKG-master/dict/check.txt ', 'w+')
        f_department = open('/Users/xieshenqiu/Downloads/知识图谱论文集/中医\ 问答系统/QASystemOnMedicalKG-master/dict/department.txt', 'w+')
        f_producer = open('/Users/xieshenqiu/Downloads/知识图谱论文集/中医\ 问答系统/QASystemOnMedicalKG-master/dict/producer.txt', 'w+')
        f_symptom = open('/Users/xieshenqiu/Downloads/知识图谱论文集/中医\ 问答系统/QASystemOnMedicalKG-master/dict/symptom.txt', 'w+')
        f_disease = open('/Users/xieshenqiu/Downloads/知识图谱论文集/中医\ 问答系统/QASystemOnMedicalKG-master/dict/disease.txt', 'w+')

        f_drug.write('\n'.join(list(Drugs)))
        f_food.write('\n'.join(list(Foods)))
        f_check.write('\n'.join(list(Checks)))
        f_department.write('\n'.join(list(Departments)))
        f_producer.write('\n'.join(list(Producers)))
        f_symptom.write('\n'.join(list(Symptoms)))
        f_disease.write('\n'.join(list(Diseases)))

        f_drug.close()
        f_food.close()
        f_check.close()
        f_department.close()
        f_producer.close()
        f_symptom.close()
        f_disease.close()

        return


if __name__ == '__main__':
    handler = MedicalGraph()
    handler.create_graphnodes()
    handler.create_graphrels()
    handler.export_data()
    print('Done')

1
'recommand_eat'
2
3
4
5
6
'recommand_eat'
7
'recommand_eat'
8
9
10
11
'recommand_eat'
12
13
14
'recommand_eat'
15
16
17
18
'recommand_eat'
19
20
21
22
23
24
25
26
27
28
29
30
'recommand_eat'
31
32
'recommand_eat'
33
34
35
36
37
38
39
40
41
42
43
'recommand_eat'
44
45
46
47
48
49
50
51
'recommand_eat'
52
53
54
55
56
57
58
'recommand_eat'
59
'recommand_eat'
60
'recommand_eat'
61
62
'recommand_eat'
63
64
'recommand_eat'
65
66
'recommand_eat'
67
68
69
'recommand_eat'
70
71
72
73
'recommand_eat'
74
75
'recommand_eat'
76
77
78
'recommand_eat'
79
80
81
82
83
84
85
'recommand_eat'
86
87
'recommand_eat'
88
89
90
91
92
'recommand_eat'
93
94
'recommand_eat'
95
96
'recommand_eat'
97
'recommand_eat'
98
99
'recommand_eat'
100
101
'recommand_eat'
102
103
104
'recommand_eat'
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
'recommand_eat'
126
'recommand_eat'
127
128
129
130
131
132
133
134
'recommand_eat'
135
136
137
138
139
140
'recommand_eat'
141
'recommand_eat'


1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
'recommand_eat'
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
'recommand_eat'
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
'recommand_eat'
1968
'recommand_eat'
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
'recommand_eat'
1981
1982
1983
1984
1985
1986
'recommand_eat'
1987
1988
1989
'recommand_eat'
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
'recommand_eat'
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
'recommand_eat'
2018
2019
2020
'recommand_eat'
2021
'recommand_eat'
2022
2023
2024
2025
'recommand_eat'
2026
2027
2028
2029
2030
2031
'recommand_eat'
2032
2033
2034
2035
'recommand_eat'
2036
2037
2038
2039
2040
2041
2042
2043
'recommand_eat'
2044
2045
2046
2047
2048
2049
2050
2051
2052


3106
3107
3108
3109
3110
3111
3112
3113
3114
3115
'recommand_eat'
3116
3117
3118
3119
3120
'recommand_eat'
3121
'recommand_eat'
3122
3123
3124
'recommand_eat'
3125
3126
'recommand_eat'
3127
3128
3129
3130
'recommand_eat'
3131
3132
3133
3134
3135
'recommand_eat'
3136
3137
3138
3139
'recommand_eat'
3140
3141
3142
3143
3144
'recommand_eat'
3145
3146
3147
3148
3149
3150
'recommand_eat'
3151
3152
3153
3154
3155
'recommand_eat'
3156
3157
3158
3159
3160
'recommand_eat'
3161
3162
3163
3164
3165
'recommand_eat'
3166
3167
3168
3169
3170
'recommand_eat'
3171
3172
3173
'recommand_eat'
3174
3175
3176
3177
3178
'recommand_eat'
3179
'recommand_eat'
3180
3181
3182
3183
3184
'recommand_eat'
3185
3186
3187
3188
3189
3190
3191
3192
3193
3194
3195
3196
'recommand_eat'
3197
3198
3199
'recommand_eat'
3200
3201
3202
'recommand_eat'
3203
3204
3205
3206
3207
3208
3209
3210
3211
3212
3213
3214
3215
'recommand_eat'
3216
3217
3218
3219
'recommand_eat'
3220
'recommand_eat'
3221
3222
'recommand_eat'
3223
3224
3225


'recommand_eat'
4009
'recommand_eat'
4010
'recommand_eat'
4011
'recommand_eat'
4012
4013
'recommand_eat'
4014
4015
'recommand_eat'
4016
4017
'recommand_eat'
4018
'recommand_eat'
4019
4020
'recommand_eat'
4021
'recommand_eat'
4022
4023
'recommand_eat'
4024
4025
'recommand_eat'
4026
'recommand_eat'
4027
4028
4029
'recommand_eat'
4030
4031
'recommand_eat'
4032
4033
4034
4035
'recommand_eat'
4036
'recommand_eat'
4037
'recommand_eat'
4038
'recommand_eat'
4039
'recommand_eat'
4040
4041
'recommand_eat'
4042
4043
4044
4045
4046
'recommand_eat'
4047
'recommand_eat'
4048
4049
'recommand_eat'
4050
'recommand_eat'
4051
4052
4053
'recommand_eat'
4054
'recommand_eat'
4055
4056
4057
'recommand_eat'
4058
'recommand_eat'
4059
4060
'recommand_eat'
4061
'recommand_eat'
4062
'recommand_eat'
4063
4064
'recommand_eat'
4065
'recommand_eat'
4066
4067
'recommand_eat'
4068
4069
4070
'recommand_eat'
4071
'recommand_eat'
4072
'recommand_eat'
4073
'recommand_eat'
4074
'recommand_eat'
4075
'recommand_eat'
4076
'rec

4636
4637
4638
'recommand_eat'
4639
4640
4641
'recommand_eat'
4642
'recommand_eat'
4643
'recommand_eat'
4644
4645
4646
'recommand_eat'
4647
'recommand_eat'
4648
4649
4650
'recommand_eat'
4651
4652
4653
'recommand_eat'
4654
4655
'recommand_eat'
4656
'recommand_eat'
4657
4658
4659
'recommand_eat'
4660
4661
'recommand_eat'
4662
4663
4664
4665
'recommand_eat'
4666
4667
'recommand_eat'
4668
'recommand_eat'
4669
4670
'recommand_eat'
4671
'recommand_eat'
4672
'recommand_eat'
4673
4674
'recommand_eat'
4675
'recommand_eat'
4676
4677
4678
'recommand_eat'
4679
4680
4681
'recommand_eat'
4682
'recommand_eat'
4683
4684
4685
'recommand_eat'
4686
'recommand_eat'
4687
4688
'recommand_eat'
4689
4690
'recommand_eat'
4691
4692
4693
'recommand_eat'
4694
4695
'recommand_eat'
4696
'recommand_eat'
4697
4698
4699
4700
4701
4702
'recommand_eat'
4703
'recommand_eat'
4704
4705
'recommand_eat'
4706
'recommand_eat'
4707
4708
'recommand_eat'
4709
'recommand_eat'
4710
4711
'recommand_eat'
4712
4713
'recommand_eat'
47

5821
'recommand_eat'
5822
5823
'recommand_eat'
5824
'recommand_eat'
5825
5826
5827
'recommand_eat'
5828
'recommand_eat'
5829
5830
'recommand_eat'
5831
5832
'recommand_eat'
5833
5834
5835
'recommand_eat'
5836
'recommand_eat'
5837
5838
'recommand_eat'
5839
'recommand_eat'
5840
5841
5842
5843
'recommand_eat'
5844
'recommand_eat'
5845
'recommand_eat'
5846
5847
5848
'recommand_eat'
5849
5850
5851
'recommand_eat'
5852
5853
'recommand_eat'
5854
'recommand_eat'
5855
5856
'recommand_eat'
5857
5858
'recommand_eat'
5859
5860
'recommand_eat'
5861
5862
'recommand_eat'
5863
'recommand_eat'
5864
'recommand_eat'
5865
'recommand_eat'
5866
'recommand_eat'
5867
5868
5869
5870
5871
'recommand_eat'
5872
5873
5874
'recommand_eat'
5875
'recommand_eat'
5876
5877
'recommand_eat'
5878
'recommand_eat'
5879
'recommand_eat'
5880
'recommand_eat'
5881
5882
5883
5884
'recommand_eat'
5885
5886
5887
'recommand_eat'
5888
'recommand_eat'
5889
5890
'recommand_eat'
5891
5892
5893
'recommand_eat'
5894
5895
'recommand_eat'
5

6603
'recommand_eat'
6604
6605
6606
6607
6608
'recommand_eat'
6609
'recommand_eat'
6610
6611
'recommand_eat'
6612
6613
6614
'recommand_eat'
6615
6616
'recommand_eat'
6617
6618
'recommand_eat'
6619
'recommand_eat'
6620
6621
'recommand_eat'
6622
'recommand_eat'
6623
'recommand_eat'
6624
'recommand_eat'
6625
6626
'recommand_eat'
6627
'recommand_eat'
6628
6629
6630
6631
'recommand_eat'
6632
6633
'recommand_eat'
6634
6635
'recommand_eat'
6636
'recommand_eat'
6637
'recommand_eat'
6638
6639
6640
'recommand_eat'
6641
'recommand_eat'
6642
6643
'recommand_eat'
6644
6645
'recommand_eat'
6646
6647
6648
6649
'recommand_eat'
6650
6651
6652
'recommand_eat'
6653
'recommand_eat'
6654
6655
6656
'recommand_eat'
6657
'recommand_eat'
6658
6659
'recommand_eat'
6660
6661
6662
'recommand_eat'
6663
'recommand_eat'
6664
6665
6666
'recommand_eat'
6667
'recommand_eat'
6668
6669
'recommand_eat'
6670
6671
'recommand_eat'
6672
'recommand_eat'
6673
6674
6675
'recommand_eat'
6676
6677
'recommand_eat'
6678
6679
'recomm

7615
'recommand_eat'
7616
7617
7618
7619
7620
7621
'recommand_eat'
7622
7623
7624
7625
7626
7627
7628
7629
7630
7631
7632
7633
7634
7635
7636
7637
7638
7639
7640
7641
7642
7643
7644
'recommand_eat'
7645
7646
7647
7648
7649
7650
7651
7652
7653
7654
7655
7656
7657
7658
7659
7660
7661
7662
7663
7664
7665
7666
'recommand_eat'
7667
7668
7669
7670
7671
'recommand_eat'
7672
7673
'recommand_eat'
7674
7675
7676
7677
7678
'recommand_eat'
7679
7680
7681
7682
7683
'recommand_eat'
7684
'recommand_eat'
7685
7686
7687
7688
7689
7690
7691
7692
7693
7694
7695
7696
'recommand_eat'
7697
7698
7699
7700
7701
7702
'recommand_eat'
7703
7704
7705
'recommand_eat'
7706
7707
7708
'recommand_eat'
7709
7710
7711
7712
7713
7714
7715
'recommand_eat'
7716
'recommand_eat'
7717
7718
7719
7720
7721
7722
7723
'recommand_eat'
7724
7725
7726
'recommand_eat'
7727
'recommand_eat'
7728
7729
'recommand_eat'
7730
'recommand_eat'
7731
7732
'recommand_eat'
7733
7734
7735
'recommand_eat'
7736
7737
'recommand_eat'
7738
'recommand_e

8653
'recommand_eat'
8654
8655
'recommand_eat'
8656
'recommand_eat'
8657
'recommand_eat'
8658
'recommand_eat'
8659
'recommand_eat'
8660
'recommand_eat'
8661
8662
'recommand_eat'
8663
8664
'recommand_eat'
8665
8666
'recommand_eat'
8667
'recommand_eat'
8668
8669
'recommand_eat'
8670
8671
'recommand_eat'
8672
8673
8674
'recommand_eat'
8675
8676
'recommand_eat'
8677
8678
'recommand_eat'
8679
8680
'recommand_eat'
8681
8682
8683
'recommand_eat'
8684
8685
8686
8687
'recommand_eat'
8688
8689
'recommand_eat'
8690
8691
'recommand_eat'
8692
'recommand_eat'
8693
'recommand_eat'
8694
'recommand_eat'
8695
8696
'recommand_eat'
8697
'recommand_eat'
8698
8699
8700
8701
8702
8703
8704
'recommand_eat'
8705
8706
'recommand_eat'
8707
8708
'recommand_eat'
8709
'recommand_eat'
8710
8711
'recommand_eat'
8712
8713
8714
'recommand_eat'
8715
8716
8717
'recommand_eat'
8718
'recommand_eat'
8719
'recommand_eat'
8720
'recommand_eat'
8721
8722
'recommand_eat'
8723
'recommand_eat'
8724
8725
'recommand_eat'
8726
'recom

ServiceUnavailable: Failed to establish connection to ('127.0.0.1', 7687) (reason [Errno 61] Connection refused)